In [1]:
#!git clone https://huggingface.co/PanaceaAI/BiomedGPT-Base-Pretrained
#!git clone --single-branch --branch feature/add_transformers https://github.com/OFA-Sys/OFA.git
#!pip install OFA/transformers

In [1]:
from transformers import OFATokenizer, OFAModel
import torch
#model_name = "BiomedGPT-Base-Pretrained"
model_name = 'PanaceaAI/BiomedGPT-Base-Pretrained'
tokenizer = OFATokenizer.from_pretrained(model_name,
        quantization_config=None,
        torch_dtype=torch.float16,
        output_attentions=True,
        output_scores=True,
        return_dict_in_generate=True)#(f"./{model_name}")

model = OFAModel.from_pretrained(model_name)#(f"./{model_name}")
model = model.to("cuda")

/gpfs/workdir/restrepoda/environments/biomed_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PanaceaAI/BiomedGPT-Base-Pretrained
<super: <class 'OFATokenizer'>, <OFATokenizer object>>


/gpfs/workdir/restrepoda/environments/biomed_venv/lib/python3.12/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
import re

import torch
from PIL import Image
from torchvision import transforms


mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
resolution = 480

patch_resize_transform = transforms.Compose([
        lambda image: image.convert("RGB"),
        transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])


In [3]:
#!gdown --id 1kojeqtJSx3MzdY3jd--XzeJrdyiFuzC8 -O example.jpg

img = Image.open('example.jpg')

txt = "What modality is used to take this image?"
inputs = tokenizer([txt], return_tensors="pt").input_ids
patch_img = patch_resize_transform(img).unsqueeze(0)


In [17]:
gen = model.generate(inputs.to("cuda"), 
                     patch_images=patch_img.to("cuda"), 
                     #num_beams=5, 
                     no_repeat_ngram_size=3, 
                     max_length=512, 
                     do_sample=False,
                     temperature=0.01,
                     output_attentions=True,
                     output_scores=True,
                     return_dict_in_generate=True)


In [6]:
gen.keys()

odict_keys(['sequences', 'scores', 'encoder_attentions', 'decoder_attentions', 'cross_attentions'])

In [7]:
results = tokenizer.batch_decode(gen['sequences'], skip_special_tokens=True)

result = results[0]
result = re.sub(r'[^\w\s]', '', result).strip()

result

'illustration is used to take this information on which the continuous feedback information is used if if no'

In [8]:
gen['sequences']

tensor([[    0, 21276,    16,   341,     7,   185,    42,   335,    15,    61,
             5, 11152,  6456,   335,    16,   341,   114,   114,   117,     2]],
       device='cuda:0')

In [9]:
gen['scores'][0].shape

torch.Size([1, 59457])

In [10]:
tokenizer.encode("No")[1]

3084

In [11]:
tokenizer("yes", add_special_tokens=False)["input_ids"][0]

10932

In [12]:
gen['scores'][0][0][tokenizer("yes", add_special_tokens=False)["input_ids"][0]]

tensor(3.9010, device='cuda:0')

In [13]:
gen['scores'][0][0][tokenizer("no", add_special_tokens=False)["input_ids"][0]]

tensor(1.9095, device='cuda:0')

In [14]:
gen['scores'][0][0][tokenizer("Yes", add_special_tokens=False)["input_ids"][0]]

tensor(-2.3328, device='cuda:0')

In [15]:
gen['scores'][0][0][tokenizer("No", add_special_tokens=False)["input_ids"][0]]

tensor(-2.3322, device='cuda:0')